In [46]:
from tesseraclassifier import habitat_classifier
from tesseraclassifier import classifier_utils
import geopandas as gpd
import tempfile

import importlib
importlib.reload(classifier_utils)
importlib.reload(habitat_classifier)


<module 'tesseraclassifier.habitat_classifier' from '/home/jk871/tessera-classification/src/tesseraclassifier/habitat_classifier.py'>

In [4]:
roi_name = "Cambridge"
dir_path = "/home/jk871/cci_workshop"

labels_json_file = f'/home/jk871/tessera-interactive-map/cambridge_labels.json'
labels = classifier_utils.load_training_labels_from_json(labels_json_file)

output_dir = f"{dir_path}/classification_outputs/{roi_name}"

In [5]:
hc = habitat_classifier.HabitatClassifier(labels, 2024)


--- Step 1: Preparing Training Data ---
Fetching list of all available tiles...


Found 146452 total available tiles             for 2024.

Processing point 1/36 at                     (52.1439, 0.0907)...
  > Match found! Point belongs to tile (52.15, 0.05).

Processing point 2/36 at                     (52.1443, 0.1440)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 3/36 at                     (52.1398, 0.1229)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 4/36 at                     (52.1409, 0.1313)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 5/36 at                     (52.1411, 0.1337)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 6/36 at                     (52.1370, 0.1060)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 7/36 at                     (52.1343, 0.1039)...
  > Match found! Point belongs to tile (52.15, 0.05).

Processing point 8/36 at                     (52.1294, 0.1093)...
  > Match found! Point belong

  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 25/36 at                     (52.1500, 0.1609)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 26/36 at                     (52.1500, 0.1670)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 27/36 at                     (52.1516, 0.1673)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 28/36 at                     (52.1514, 0.1613)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 29/36 at                     (52.1562, 0.1604)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 30/36 at                     (52.1522, 0.1781)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 31/36 at                     (52.1599, 0.1506)...
  > Match found! Point belongs to tile (52.15, 0.15).

Processing point 32/36 at                     (52.1706, 0.1054)...
  > Match found! Point b

In [6]:
hc.train()


--- Step 2: Training Model ---
Training model on 36 samples...
✅ Model training complete.


In [17]:
# Classification dry run - just to estimate the size of each ROI
hc.classify_tiles(None, hc.training_tiles, max_workers=8, dry_run=True)
hc.classify_region(None, classifier_utils.roi_from_points(labels, 20), max_workers=8, dry_run=True)

Found 2 tiles to process.
Projecting points to a suitable local CRS: EPSG:32631
Found 48 tiles to process.


In [11]:
key = "training_tiles"
temp_dir = tempfile.mkdtemp(prefix=f"tessera_classify_{roi_name}_{key}_")
hc.classify_tiles(temp_dir, hc.training_tiles, max_workers=8)
print(temp_dir)

Found 2 tiles to process.

Starting parallel classification using up to 8 CPU cores...

Parallel processing complete. 2 of 2 tiles were successfully classified.
Intermediate classified tiles saved in: /tmp/tessera_classify_Cambridge_training_tiles_5e3qtkca
/tmp/tessera_classify_Cambridge_training_tiles_5e3qtkca


In [12]:
hc.mosaic_classification(temp_dir, f"{output_dir}/{key}.tif", cleanup=True)


Stitching 2 tiles into a master mosaic...
✅ Successfully created final classification map: /home/jk871/cci_workshop/classification_outputs/Cambridge/training_tiles.tif
Cleaning up temporary directory: /tmp/tessera_classify_Cambridge_training_tiles_5e3qtkca


In [31]:
key = "bounding_box"
temp_dir = tempfile.mkdtemp(prefix=f"tessera_classify_{roi_name}_{key}_")
hc.classify_region(temp_dir, classifier_utils.roi_from_points(labels, 20), max_workers=30)
print(temp_dir)

Projecting points to a suitable local CRS: EPSG:32631
Found 48 tiles to process.

Starting parallel classification using up to 30 CPU cores...

Parallel processing complete. 48 of 48 tiles were successfully classified.
Intermediate classified tiles saved in: /tmp/tessera_classify_Cambridge_bounding_box_s46lks0j
/tmp/tessera_classify_Cambridge_bounding_box_s46lks0j


In [47]:
hc.mosaic_classification(temp_dir, f"{output_dir}/{key}.tif", cleanup=False)

LALAL
False
Using EPSG:32630 as the common projection for stitching.
Starting parallel reprojection of 48 tiles...

Stitching 48 aligned tiles into final map...
✅ Successfully created final classification map: /home/jk871/cci_workshop/classification_outputs/Cambridge/bounding_box.tif
